### CSC478 Final Project: MovieLens
### Jun Tae Son |  Yuchen Wu | Tzu Hao Peng

In [243]:
import os
import math
import pandas as pd
import numpy as np
from numpy import *
from numpy import linalg as la
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error, mean_absolute_error, jaccard_similarity_score
from sklearn import preprocessing
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.cross_validation import train_test_split


# change the directory
os.chdir('C:\\Users\\rkfql\\Desktop\\Movielens-02')

C:\Users\rkfql\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# read rating data
rating_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=rating_cols)
print(ratings.shape)
ratings.head()

(100000, 4)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
# read movie data
item_cols = ['movie_id','movie_title','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film_Noir','Horror','Musical','Mystery','Romance','Sci_Fi','Thriller','War' ,'Western']
movies=pd.read_csv('u.item',sep='|',encoding='iso-8859-1', names=item_cols)
print(movies.shape)
movies.head()

(1682, 24)


,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
# read user data
user_cols = ['user_id','age','gender','occupation','zip_code']
users = pd.read_csv('u.user', sep='|', names=user_cols)
print(users.shape)
users.head()

(943, 5)


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


# Genre matrix

<br> rows = unique movie_id </br>
<br> cols = unique genre </br>

In [117]:
genre_mat = movies.iloc[:,5:].T
print(genre_mat.shape)
genre_index = list(genre_mat.index.values)
genre_mat.head()

(19, 1682)


,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
unknown,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Action,0,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Adventure,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Animation,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Childrens,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# User Profile

In [231]:
# reformat rating matrix to be one row per column and one column per user.
df = ratings.pivot(index = 'movie_id', columns ='user_id', values = 'rating').fillna(0)
print(df.shape)
df.head()

(1682, 943)


user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
2,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
# calculate the dot product of the genre matrix and the ratings matrix and obtain the user profiles.

genre_array = np.array(genre_mat)
df_array = np.array(df)

user_profile = np.dot(genre_array,df_array)
mm = preprocessing.MinMaxScaler().fit(user_profile)
user_profile_norm = mm.fit_transform(user_profile)
print(user_profile_norm.shape)

df_user_profile = pd.DataFrame(user_profile_norm,index = genre_index)
df_user_profile.head()

(19, 943)


,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
unknown,0.000000,0.000000,0.000000,0.000000,0.016260,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Action,0.591346,0.283582,0.609375,0.720930,0.715447,0.216495,0.600000,1.000000,0.454545,0.309249,...,0.550388,0.845070,0.338235,0.252874,1.000000,0.890244,0.456647,1.000000,0.540146,1.000000
Adventure,0.286058,0.097015,0.218750,0.325581,0.434959,0.195876,0.377236,0.408805,0.340909,0.176301,...,0.379845,0.422535,0.158088,0.137931,0.533898,0.426829,0.208092,0.710526,0.379562,0.502203
Animation,0.086538,0.029851,0.000000,0.000000,0.215447,0.087629,0.094309,0.000000,0.000000,0.083815,...,0.085271,0.042254,0.014706,0.057471,0.161017,0.048780,0.052023,0.368421,0.138686,0.030837
Childrens,0.122596,0.089552,0.000000,0.000000,0.288618,0.157216,0.237398,0.025157,0.000000,0.098266,...,0.205426,0.042254,0.044118,0.068966,0.305085,0.048780,0.121387,0.236842,0.408759,0.110132


- positive values shows a preference towards a certain genre.


# Content_based recommender

retrieve movies that are closest in similarity to a user’s profile, which represents a user’s preference for an item’s feature.

In [256]:
# user-id = 0
# This user strongly prefers watching Drama, Romance, and Sci-fi genre movies.
sample = df_user_profile.iloc[:,1]
sample

unknown        0.000000
Action         0.283582
Adventure      0.097015
Animation      0.029851
Childrens      0.089552
Comedy         0.455224
Crime          0.253731
Documentary    0.000000
Drama          1.000000
Fantasy        0.022388
Film_Noir      0.067164
Horror         0.044776
Musical        0.022388
Mystery        0.104478
Romance        0.492537
Sci_Fi         0.111940
Thriller       0.320896
War            0.082090
Western        0.000000
Name: 1, dtype: float64

In [257]:
genre_mat[0]

unknown        0
Action         0
Adventure      0
Animation      1
Childrens      1
Comedy         1
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film_Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci_Fi         0
Thriller       0
War            0
Western        0
Name: 0, dtype: int64

In [258]:
# movie dic
temp = movies.iloc[:,:2]
movie_dic = {}
for i in range(temp.shape[0]):
    movie_dic[temp.movie_id[i]] = temp.movie_title[i]

In [259]:
def ecludSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = inA.T * inB
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

#jaccard_similarity_score(genre_mat[i],sample)

In [260]:
# calculate pearson similarity
sim_dic = {}
for i in range(genre_mat.shape[1]):
    sim = pearsSim(genre_mat[i],sample)
    sim_dic[i+1] = sim

In [261]:
sim_dic

{1: 0.50759860994057693,
 2: 0.54535785626067435,
 3: 0.5670057815918722,
 4: 0.8540952232308836,
 5: 0.80523031622840413,
 6: 0.897071298322206,
 7: 0.76377092340002384,
 8: 0.79634578768249886,
 9: 0.897071298322206,
 10: 0.75321453046774267,
 11: 0.57375585061896406,
 12: 0.57375585061896406,
 13: 0.63229346621985039,
 14: 0.89836493328660794,
 15: 0.897071298322206,
 16: 0.70571076227247775,
 17: 0.60919468332216142,
 18: 0.897071298322206,
 19: 0.897071298322206,
 20: 0.89836493328660794,
 21: 0.56505215176639401,
 22: 0.74303861640706714,
 23: 0.83766567392599156,
 24: 0.52536766703238746,
 25: 0.63229346621985039,
 26: 0.63229346621985039,
 27: 0.5488703136396561,
 28: 0.81411484477430951,
 29: 0.59514964682515925,
 30: 0.897071298322206,
 31: 0.75414427708944876,
 32: 0.41104075720281352,
 33: 0.66307785949391951,
 34: 0.88516944212125659,
 35: 0.68973144513163531,
 36: 0.89836493328660794,
 37: 0.897071298322206,
 38: 0.52361298419062874,
 39: 0.52980993130534015,
 40: 0.63229

In [270]:
sorted_sim = Counter(sim_dic)
for key, value in sorted_sim.most_common(10):
    print(movie_genre_dic[key])
    print('%s' % movie_dic[key], '\n')

['Comedy', 'Drama', 'Romance']
Cinema Paradiso (1988) 

['Comedy', 'Drama', 'Romance']
Wings of Desire (1987) 

['Comedy', 'Drama', 'Romance']
Manhattan (1979) 

['Comedy', 'Drama', 'Romance']
American President, The (1995) 

['Comedy', 'Drama', 'Romance']
Corrina, Corrina (1994) 

['Comedy', 'Drama', 'Romance']
Something to Talk About (1995) 

['Comedy', 'Drama', 'Romance']
Don Juan DeMarco (1995) 

['Comedy', 'Drama', 'Romance']
Brassed Off (1996) 

['Comedy', 'Drama', 'Romance']
What Happened Was... (1994) 

['Comedy', 'Drama', 'Romance']
Twelfth Night (1996) 



# Dictionary for analysis

In [17]:
# make genre dictionary
genre = pd.read_csv('u.genre', sep='|', names = ['genre','key'])
genre = np.array(genre.iloc[:,0])

genre_dic = {}
for i in range(len(genre)):
    genre_dic[i] = genre[i]

genre_dic

{0: 'unknown',
 1: 'Action',
 2: 'Adventure',
 3: 'Animation',
 4: "Children's",
 5: 'Comedy',
 6: 'Crime',
 7: 'Documentary',
 8: 'Drama',
 9: 'Fantasy',
 10: 'Film-Noir',
 11: 'Horror',
 12: 'Musical',
 13: 'Mystery',
 14: 'Romance',
 15: 'Sci-Fi',
 16: 'Thriller',
 17: 'War',
 18: 'Western'}

In [56]:
temp = pd.DataFrame.copy(movies)
temp = temp.drop(['movie_title','release_date','video_release_date','IMDb_URL'], axis=1)
temp = np.array(temp)

movie_dic = {}
for i in range(len(temp)):
    movie_dic[temp[i][0]] = temp[i][1:]

movie_genre = []
for j in range(len(movie_dic)):
    entry_genre=[]
    for k in range(len(movie_dic[j+1])):
        if movie_dic[j+1][k] != 0:
            entry_genre.append(genre_dic[k])
    movie_genre.append(entry_genre)

In [57]:
movie_genre_dic = {}
for i in range(len(temp)):
    movie_genre_dic[temp[i][0]] = movie_genre[i]

In [63]:
temp_ratings = pd.DataFrame.copy(ratings)
movie_id_df = temp_ratings.iloc[:,1]

genre_lst = []
for i in range(len(movie_id_df)):
    index = movie_id_df[i]
    genre_lst.append(movie_genre_dic[index])

In [65]:
# join ratings and movie_genre_dic
genre_temp = pd.DataFrame({'movie_genre':genre_lst})
genre_temp['Joined'] = genre_temp.movie_genre.apply(', '.join)

temp_ratings['genre'] = genre_temp.Joined
temp_ratings.head()

,user_id,movie_id,rating,timestamp,genre
0,196,242,3,881250949,Comedy
1,186,302,3,891717742,"Crime, Film-Noir, Mystery, Thriller"
2,22,377,1,878887116,"Children's, Comedy"
3,244,51,2,880606923,"Drama, Romance, War, Western"
4,166,346,1,886397596,"Crime, Drama"
